<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/tf_quant_finance_notebooks/longstaff_schwartz_least_square_mc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Values Amercian style options using the LSM algorithm.


In [1]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

In [2]:
!pip install tf-quant-finance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 47.2 MB/s eta 0:00:00


In [8]:
import numpy as np
import tf_quant_finance as tff

from tf_quant_finance.experimental.lsm_algorithm import make_basket_put_payoff
from tf_quant_finance.experimental.lsm_algorithm import make_polynomial_basis
from tf_quant_finance.experimental.lsm_algorithm import least_square_mc

## tf_quant_finance.models.longstaff_schwartz.make_polynomial_basis

In [11]:
basis = tff.experimental.lsm_algorithm.make_polynomial_basis_v2(2)
x = [[1.0], [2.0], [3.0], [4.0]]
x = tf.expand_dims(x, axis=-1)
basis(x, tf.constant(0, dtype=np.int32))
# Expected result:
[[ 1.  ,  1.  ,  1.  ,  1.  ], [-1.5 , -0.5 ,  0.5 ,  1.5 ],
[ 2.25,  0.25,  0.25,  2.25]]

[[1.0, 1.0, 1.0, 1.0], [-1.5, -0.5, 0.5, 1.5], [2.25, 0.25, 0.25, 2.25]]

In [10]:
# Let the underlying model be a Black-Scholes process
# dS_t / S_t = rate dt + sigma**2 dW_t, S_0 = 1.0
# with `rate = 0.1`, and volatility `sigma = 1.0`.
# Define drift and volatility functions for log(S_t)
rate = 0.1
def drift_fn(_, x):
  return rate - tf.ones_like(x) / 2.
def vol_fn(_, x):
  return tf.expand_dims(tf.ones_like(x), axis=-1)
# Use Euler scheme to propagate 100000 paths for 1 year into the future
times = np.linspace(0., 1, num=50)
num_samples = 100000
log_paths = tf.function(tff.models.euler_sampling.sample)(
        dim=1,
        drift_fn=drift_fn, volatility_fn=vol_fn,
        random_type=tff.math.random.RandomType.PSEUDO_ANTITHETIC,
        times=times, num_samples=num_samples, seed=42, time_step=0.01,
        dtype=tf.float64) # Specify dtype as tf.float64
# Compute exponent to get samples of `S_t`
paths = tf.math.exp(log_paths)
# American put option price for strike 1.1 and expiry 1 (assuming actual day
# count convention and no settlement adjustment)
strike = [1.1]
exercise_times = tf.range(times.shape[-1])
discount_factors = tf.exp(-rate * times)
payoff_fn = make_basket_put_payoff(strike, dtype=tf.float64) # Specify dtype as tf.float64
basis_fn = make_polynomial_basis(10)
least_square_mc(paths, exercise_times, payoff_fn, basis_fn,
                discount_factors=discount_factors)
# Expected value: [0.397]
# European put option price
tff.black_scholes.option_price(volatilities=[1], strikes=strike, # Change strikes to strike
                               expiries=[1], spots=[1.],
                               discount_factors=discount_factors[-1],
                               is_call_options=False,
                               dtype=tf.float64)
# Expected value: [0.379]

<tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.37969354])>